<a href="https://colab.research.google.com/github/JuniorHZ19/ML/blob/main/perceptronsimplepytorch2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio

In [2]:
import torch
import numpy as np


arraglo_numpy_float64=np.array([[1,2,3,4],
                                [5,3,8,7],
                                [7,2,3,4]],dtype=np.float64)


tensor_trasnformado=torch.from_numpy(arraglo_numpy_float64)

tensor_trasnformado

tensor([[1., 2., 3., 4.],
        [5., 3., 8., 7.],
        [7., 2., 3., 4.]], dtype=torch.float64)

In [3]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import sklearn
from sklearn.preprocessing import StandardScaler



class MiDataSet(Dataset):

  def __init__(self,csv_file):

     data=pd.read_csv(csv_file)

     self.x,self.y= self.limpiezaDatos(data)

     self.samples=self.y.shape[0]


  def __getitem__(self,id):

   return self.x[id],self.y[id]


  def __len__(self):
   return self.samples


  def limpiezaDatos(self,data):

    data=data.dropna(how="all")

    data=data.drop_duplicates()

    data=data.drop(columns=["name","ticket","cabin","boat","body","home.dest"])

    data[["age","fare"]]=data[["age","fare"]].fillna(data[["age","fare"]].mean())

    data["embarked"]=data["embarked"].fillna("S")

    data["age"]=data["age"].astype(int)

    data=pd.get_dummies (data,columns=["sex","embarked"],prefix=["sex","embarked"])

    variables_x=data.drop(columns=["survived"])

    print(variables_x.head())

    print(data["survived"].head())
    self.scaler=StandardScaler()

    variables_x_esclados=self.scaler.fit_transform(variables_x)

    variable_y=data["survived"].values


    datos_x=torch.from_numpy(variables_x_esclados).float()

    datos_y=torch.from_numpy(variable_y).float()



    return datos_x,  datos_y







In [ ]:
import sklearn

from sklearn.model_selection import train_test_split

mis_datos=MiDataSet("/content/titanic.csv")



X_entrenamiento, X_prueba, y_entrenamiento, y_prueba = train_test_split(mis_datos.x, mis_datos.y, test_size=0.2, random_state=0)

print(X_entrenamiento.shape)

print(X_prueba.shape)

print(y_entrenamiento.shape)

print(y_prueba.shape)




In [ ]:


from torch.utils.data import DataLoader, TensorDataset


conjunto_entrenamiento = TensorDataset(X_entrenamiento, y_entrenamiento)

conjunto_prueba = TensorDataset(X_prueba, y_prueba)



data_por_lote_entrenamiento=DataLoader(conjunto_entrenamiento,batch_size=100,shuffle=True)

data_por_por_lote_test=DataLoader(conjunto_prueba,batch_size=100,shuffle=True)



datos_features_train,datos_label_train=next(iter(data_por_lote_entrenamiento))

datos_features_test,datos_label_test=next(iter(data_por_por_lote_test))


print(datos_features_test.shape)




In [5]:
import torch.nn as nn

class Perceptron(nn.Module):

 def __init__(self,input_size):
  super(Perceptron,self).__init__()

  self.capalineal=nn.Linear(input_size,1)

 def forward(self,x):
    salidaNeurona=torch.sigmoid(self.capalineal(x))
    return salidaNeurona







In [ ]:
import torch.optim as optim
from sklearn.metrics import accuracy_score

input_size=X_entrenamiento.shape[1]

PerceptronSimple=Perceptron(input_size)

criterio=nn.BCELoss()

optimizer=optim.SGD(PerceptronSimple.parameters(),lr=0.001)

epochs=19000

#funcion clasificador:
def clasificador(y_pred_test):

  umbral = 0.7
  y_pred_test_binario = (y_pred_test >= umbral).to(torch.float32)

  return y_pred_test_binario

#funcion acutity:
def accurity(y_test,y_real):
   correctos=0
   for predicho, real in zip(y_test, y_real):
    if predicho==real:
     correctos+=1
   return correctos/len(y_test)


for epoch in range(epochs):

  y_predict=PerceptronSimple(X_entrenamiento)

  loss=criterio(y_predict,y_entrenamiento.view(-1,1))

  loss.backward()

  optimizer.step()


  optimizer.zero_grad()

  with torch.no_grad():
   y_predict_test=PerceptronSimple(X_prueba)
   y_predic_test_clasificado=clasificador(y_predict_test)



   Precision=accurity(y_prueba,y_predic_test_clasificado)



  print(f'Epoca[{epoch+1}],Perdida: {loss.item():.4f} ,Accuracy: {Precision:.4f}')


#nuevo datos:

def escalarNuevaData(data):

    scaler=StandardScaler()

    datos_escalados=scaler.fit_transform(data)

    datos_x_tensor=torch.from_numpy(datos_escalados).float()

    return datos_x_tensor

nuevo_x=escalarNuevaData([[1,29,0,0,211,1,0,0,0,1]])

y_predict=PerceptronSimple(nuevo_x)

print(clasificador(y_predict))

In [11]:


y_pred_multiclases = torch.tensor([[0.1, 1.8, 0.8, 0.4, 2],
                                   [0.5, 2, 3.5, 5, 2.1]])

def clasificador(y_pred_test):

   max_values_tensor = torch.empty((0,), dtype=torch.float32)

   for tensor in y_pred_test:

     max_values, max_indices  = torch.max(tensor, dim=0)

     max_values_tensor = torch.cat((max_values_tensor, max_indices.unsqueeze(0)))

   return max_values_tensor

print(clasificador(y_pred_multiclases))


tensor([4., 3.])
